In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime 
import os
from datetime import timedelta
from constants import *
from trip_file import TripFile
from trip import Trip


### Read all files

In [2]:
taarif = pd.read_csv("../files/taarif.csv")
taarif = taarif.drop(0).reset_index().drop(columns=["index"]) # first row is nulls
new_drivers = pd.read_csv("../files/new_drivers.csv", index_col=0)
drivers_with_kviut = pd.read_csv("../files/drivers_with_kviut.csv",index_col=0)

In [3]:
def preprocess_drivers(drivers: pd.DataFrame, copy=False) -> pd.DataFrame:
    df = drivers
    if copy:
        df = drivers.copy()

    gender_mapping = {"F": FEMALE,
                    "M":MALE,
                    "m":MALE,
                    "male":MALE,
                    "boy":MALE,
                    "unknown":UNKOWN,
                    'woman':FEMALE,
                    'girl':FEMALE,
                    'none':UNKOWN,
                    'female':FEMALE,
                    UNKOWN:UNKOWN}

    # Make preprocess
    df.gender = df.gender.fillna(UNKOWN)
    df.gender = df.gender.apply(lambda x: gender_mapping[x])
    df.birthdate = pd.to_datetime(df.birthdate, format="mixed")
    drivers.birthdate = drivers.birthdate.fillna(datetime.datetime.now())

    return df

def preprocess_taarif(taarif_df: pd.DataFrame, copy=False) -> pd.DataFrame:
    df = taarif_df
    if copy:
        df = new_drivers.copy()

    customers_mapping = {
            "dbs" : "yes",
            "mizranei_kfar_saba": "aminach",
            "telecommunication_ltd": "hot" 
    }

    for customer, new_customer in customers_mapping.items():
        # Select the row to duplicate (let's say the first row, index 0)
        row_to_duplicate = df[df["customer"] == customer]
        # Make a copy of the row
        new_row = row_to_duplicate.copy()
        new_row["customer"] = new_customer
        # Append the modified row to the DataFrame
        df = pd.concat([df,pd.DataFrame(new_row)], ignore_index=True)
        
    return df


# Concat the two tables of drivers with and without kviut
def concat_drivers_tables(new_drivers, drivers_with_kviut):
    drivers_with_kviut["kviut"] = 1
    drivers = pd.concat([new_drivers, drivers_with_kviut])
    drivers["kviut"] = drivers["kviut"].fillna(0)
    return drivers
    

In [4]:
drivers = concat_drivers_tables(new_drivers, drivers_with_kviut)
drivers = preprocess_drivers(drivers, copy=True)
taarif = preprocess_taarif(taarif)

In [10]:
from tqdm import tqdm
def get_trip_files(folder, limit=100):
    files = np.array(list(os.listdir(folder)))
    if limit:
        files = files[:limit]
    files = np.array([name if not name[-5] == ")" else name[:-7]+".csv" for name in files])
    print("Processing files....")
    return np.array([TripFile(os.path.join(folder,file)) for file in tqdm(files)])


def is_friday(time):
    return time.weekday() == 4
def is_after_4pm(time):
    return time.hour >=16
def is_saturday(time):
    return time.weekday() == 5
def is_before_8pm(time):
    return time.hour < 20 or (time.hour == 20 and time.minute == 0 and time.second == 0) 

def is_after_hour(time, hour):
    return time.hour >= hour
def is_before_hour(time, hour):
    return time.hour < hour or (time.hour == hour and time.minute == 0 and time.second == 0) 

def count_time_in_weekend(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(hours=1))
        if is_friday(next_time):
            if is_after_hour(next_time, 16):
                friday_4pm = datetime.datetime(next_time.year, next_time.month, next_time.day, 16, 0)
                count += (next_time - max(current_time, friday_4pm)) / timedelta(hours=1)
        elif is_saturday(next_time):
            if is_before_hour(current_time, 20) or is_friday(current_time): # when next time jumps to midnight the current time is still on friday
                saturday_8pm= datetime.datetime(next_time.year, next_time.month, next_time.day, 20, 0)
                count += (min(next_time, saturday_8pm) - current_time) / timedelta(hours=1)
        
        current_time = current_time + timedelta(hours=1)

    return count

def count_drive_weekend_time(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(days=1))
        if is_friday(current_time) or is_saturday(current_time):
            if is_friday(current_time):
                weekend_start = datetime.datetime(current_time.year, current_time.month, current_time.day, 22, 0)
                weekend_end = datetime.datetime(current_time.year, current_time.month, current_time.day, 16, 0)
            else:
                weekend_start = datetime.datetime(current_time.year, current_time.month, current_time.day, 22, 0)
                weekend_end = datetime.datetime(current_time.year, current_time.month, current_time.day, 22, 0)
            interval1 = (start_time, end_time)
            # interval2 = (night_start, night_end)
            # count += overlap_hours(interval1, interval2)

    return count



def count_time_at_night(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(hours=1))
        if is_after_hour(next_time, 22):
                ten_pm = datetime.datetime(next_time.year, next_time.month, next_time.day, 22, 0)
                count += (next_time - max(current_time, ten_pm)) / timedelta(hours=1)
        elif is_before_hour(current_time, 6) or is_after_hour(current_time, 22):
                six_am = datetime.datetime(next_time.year, next_time.month, next_time.day, 6, 0)
                count += (min(next_time, six_am) - current_time) / timedelta(hours=1)
        
        current_time = current_time + datetime.timedelta(hours=1)
    return count


def count_drive_night_time(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(days=1))
        night_start = datetime.datetime(current_time.year, current_time.month, current_time.day, 22, 0)
        night_end = night_start + timedelta(hours=8)
        interval1 = (start_time, end_time)
        interval2 = (night_start, night_end)
        count += overlap_hours(interval1, interval2)

    return count

        

def overlap_hours(interval1, interval2):
    start1, end1 = interval1
    start2, end2 = interval2

    # Check if intervals overlap
    if start1 <= end2 and start2 <= end1:
        # Calculate the overlap duration
        overlap_start = max(start1, start2)
        overlap_end = min(end1, end2)
        overlap_duration = overlap_end - overlap_start

        return overlap_duration.total_seconds() / 3600
    else:
        return 0


def count_time_at_night_and_weekend(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + timedelta(hours=1))
        if is_after_hour(next_time, 22):
            if is_friday(next_time):
                ten_pm = datetime.datetime(next_time.year, next_time.month, next_time.day, 22, 0)
                count += (next_time - max(current_time, ten_pm)) / timedelta(hours=1)
        elif (is_before_hour(current_time, 6) or is_after_hour(current_time, 22)):
            if is_saturday(next_time):
                six_am = datetime.datetime(next_time.year, next_time.month, next_time.day, 6, 0)
                count += (min(next_time, six_am) - current_time) / timedelta(hours=1)
        
        current_time = current_time + timedelta(hours=1)
    return count

# Calculate age function
def calculate_age(birth_date):
    current_date = datetime.datetime.now()
    if pd.isnull(birth_date):
        return np.nan
    age = current_date.year - birth_date.year - ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
    return age

In [7]:
trip_files = get_trip_files("../files/trips_data", limit=1000)


Processing files....


100%|██████████| 1000/1000 [00:06<00:00, 144.90it/s]


In [15]:
from time import time
def calculate_trips_cost(trips_with_fares):
    
    start = time()
    df = trips_with_fares.copy()
    print("copying time ", time() - start)
    
    start = time()
    
    night_and_weekend_hours =  df.apply(lambda r: count_time_at_night_and_weekend(r.start_time, r.end_time), axis=1)
    print("calculate night and weekend time ", time() - start)
    start = time()

    weekend_hours = df.apply(lambda r: count_time_in_weekend(r.start_time, r.end_time), axis=1) - night_and_weekend_hours
    print("calculate weekend time ", time() - start)
    start = time()

    # night_hours2 = df.apply(lambda r: count_drive_night_time(r.start_time, r.end_time), axis=1) - night_and_weekend_hours
    night_hours = df.apply(lambda r: count_time_at_night(r.start_time, r.end_time), axis=1) - night_and_weekend_hours
    # print((night_hours == night_hours2))
    # print(night_hours, night_hours2)
    print("calculate night time ", time() - start)


    start = time()
    km_per_hour = df["km"] / ((df["end_time"]-df["start_time"]) / timedelta(hours=1))
    print("calculate kph ", time() - start)
    
    start = time()
    basic_pay = df.km * df.basic_taarif
    extra_pay = df.km.apply(lambda km: max(0, km - 200)) * df.extra_milage
    payment_without_bonus = basic_pay + extra_pay
    payment_per_km = payment_without_bonus / df["km"]
    print("calculate basic payments ", time() - start)


    start = time()
    # bonus_precentage = ((weekend_hours * df.weekend_bonus) + (night_hours * df.night_bonus) + (night_and_weekend_hours * (df.weekend_bonus + df.night_bonus)))
    night_extra_payment = (df.night_bonus / 100) * ((km_per_hour * night_hours) * payment_per_km)
    weekend_extra_payment = (df.weekend_bonus / 100) * ((km_per_hour * weekend_hours) * payment_per_km)
    night_and_weekend_extra_payment = ((df.weekend_bonus + df.night_bonus) / 100) * ((km_per_hour * night_and_weekend_hours) * payment_per_km)
    payment_with_bonus = payment_without_bonus + night_extra_payment + weekend_extra_payment + night_and_weekend_extra_payment
    print("calculate bonuses ", time() - start)

    df["basic_pay"] = basic_pay
    df["extra_milage_pay"] = extra_pay
    df["weekend_hours"] = weekend_hours
    df["night_hours"] = night_hours
    df["night_and_weekend_hours"] = night_and_weekend_hours
    df["kph"] = km_per_hour
    df["weekend_km"] = weekend_hours * km_per_hour
    df["night_extra_payment"] = night_extra_payment
    df["weekend_extra_payment"] = weekend_extra_payment
    df["night_and_weekend_extra_payment"] = night_and_weekend_extra_payment
    df["bonus_payment"] = night_extra_payment + weekend_extra_payment + night_and_weekend_extra_payment
    df["payment_without_bonus"] = payment_without_bonus
    df["payment_with_bonus"] = payment_with_bonus
    
    return df


def create_main_table(trip_files=None):
    cum_table = pd.DataFrame(columns=["driver_id","month","year","total_income","total_km"])
    if trip_files is None:
        trip_files = get_trip_files("../files/trips_data", limit=100)
    print("Processing income of drivers...")
    for trip_file in tqdm(trip_files):
        
        trip_df = trip_file.get_data_frame()
        trips_with_fares = pd.merge(trip_df, taarif, on=["customer"],how="left")
        df = calculate_trips_cost(trips_with_fares)
        df["month"] = trip_file.get_month()
        df["year"] = trip_file.get_year()

        # print(df)
        subset_columns = ["driver_id", "month", "year", "payment_with_bonus", "km"]
        df_to_concat = df[subset_columns]
        new_column_names = {'payment_with_bonus': 'total_income', 'km': 'total_km'}
        df_to_concat = df_to_concat.rename(columns=new_column_names)
        
        cum_table = pd.concat([cum_table, df_to_concat], axis=0, join="outer")
        cum_table = cum_table.reset_index().drop(columns=["index"])
        cum_table = cum_table.groupby(["driver_id","month","year"]).sum().reset_index()

        drivers_copy = drivers.copy()
        drivers_copy["age"] = drivers.birthdate.apply(calculate_age)
        drivers_copy = drivers_copy.rename(columns={"id":"driver_id"})
        
    cum_table = pd.merge(cum_table, drivers_copy[["driver_id","gender","age","vetek","kviut"]], on=["driver_id"], how="left")

    return cum_table


def create_main_table2(trip_files=None):
    cum_table = pd.DataFrame(columns=["driver_id","month","year","total_income","total_km"])
    if trip_files is None:
        trip_files = get_trip_files("../files/trips_data", limit=100)

    all_trips_dfs = [trip_file.get_data_frame() for trip_file in trip_files]  
    # print(all([type(df) == pd.DataFrame for df in all_trips_dfs]))

    df_trips = pd.concat(all_trips_dfs, axis=0, join="outer")
    trips_with_fares = pd.merge(df_trips, taarif, on=["customer"],how="left")
    df = calculate_trips_cost(trips_with_fares)

    

    return cum_table

table = create_main_table2(trip_files=trip_files[:1000])


copying time  0.0
calculate night and weekend time  6.174329519271851
calculate weekend time  6.7923195362091064
calculate night time  6.753960132598877
calculate kph  0.0
calculate basic payments  0.015570640563964844
calculate bonuses  0.01564192771911621


In [52]:
table

,driver_id,month,year,total_income,total_km,gender,age,vetek,kviut
0,1,april,2015,31308.948080,1790.648793,f,41.0,1.4,1.0
1,2,april,2015,19494.803449,1294.412410,m,63.0,1.0,1.0
2,3,april,2015,16022.174074,1114.230599,m,47.0,4.7,1.0
3,4,april,2015,103839.382196,4355.427576,m,58.0,11.7,1.0
4,5,april,2015,24435.203219,960.505611,m,38.0,4.7,1.0
...,...,...,...,...,...,...,...,...,...
990,996,april,2015,105989.998624,5343.378751,m,40.0,175.0,0.0
991,997,april,2015,25506.580995,2170.379394,m,60.0,10.5,1.0
992,998,april,2015,105668.034412,4609.298786,m,30.0,1.4,1.0
993,999,april,2015,65820.043002,2342.700295,m,43.0,14.1,1.0


In [ ]:
def count_time_at_night_and_weekend(start_time, end_time):
    count = 0
    current_time = start_time

    while current_time < end_time:
        next_time = min(end_time, current_time + datetime.timedelta(hours=1))
        if is_after_hour(next_time, 22):
            if is_friday(next_time):
                ten_pm = datetime.datetime(next_time.year, next_time.month, next_time.day, 22, 0)
                count += (next_time - max(current_time, ten_pm)) / timedelta(hours=1)
        elif (is_before_hour(current_time, 6) or is_after_hour(current_time, 22)):
            if is_saturday(next_time):
                six_am = datetime.datetime(next_time.year, next_time.month, next_time.day, 6, 0)
                count += (min(next_time, six_am) - current_time) / timedelta(hours=1)
        
        current_time = current_time + datetime.timedelta(hours=1)
    return count


# data = pd.DataFrame({
#     'start_time': ['2024-05-03 23:15:00', '2024-05-04 17:00:00', '2024-05-02 04:00:00', '2024-05-03 16:45:00'],
#     'end_time': ['2024-05-04 02:30:00', '2024-05-04 19:30:00', '2024-05-05 08:00:00', '2024-05-04 19:15:00']
# })

data = pd.DataFrame({
    'start_time': ['2024-05-03 23:00:00','2024-05-03 16:45:00'],
    'end_time': [ '2024-05-11 03:00:00','2024-05-04 03:30:00']
})

def count_this(r):
    return count_time_at_night_and_weekend(r["start_time"],r["end_time"])
    # return count_time_in_weekend(r["start_time"],r["end_time"])
    
# data = data.map(datetime.datetime)
data.start_time = pd.to_datetime(data.start_time)
data.end_time = pd.to_datetime(data.end_time)
data.apply(count_this, axis=1)

0    12.0
1     5.5
dtype: float64